In [2]:
import pygsheets
def write_to_gsheet(service_file_path, spreadsheet_id, sheet_name, data_df):
    """
    this function takes data_df and writes it under spreadsheet_id
    and sheet_name using your credentials under service_file_path
    """
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
    try:
        sh.add_worksheet(sheet_name)
    except: 
        pass
    wks_write = sh.worksheet_by_title(sheet_name)
    wks_write.clear('A1',None,'*')
    wks_write.set_dataframe(data_df, (1,1), encoding='utf-8', fit=True)
    wks_write.frozen_rows = 1

import pandas as pd

# data = pd.read_csv('add2.csv')

from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google.oauth2 import service_account


SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SERVICE_ACCOUNT_FILE = 'keys.json'

creds = None
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

In [2]:
SAMPLE_SPREADSHEET_ID = '1r8RzjvCB4VLzYr9nVSO71R3vwbmMVPpEHkLKx2gBrmE'

service = build('sheets', 'v4', credentials=creds)

# Call the Sheets API
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,range="raw!A:B").execute()
values_1 = result.get('values', [])

In [3]:
data = pd.DataFrame(values_1)
new_header = data.iloc[0] #grab the first row for the header
data = data[1:] #take the data less the header row
data.columns = new_header #set the header row as the df header

In [5]:
data.head()

,customer_id,Address
0,3300924,house no. 129 Suradevi Bus stop kampthee road...
1,996216,14 mayuresh Ring road behind UCO bank sneh Nag...
2,3063295,Darshan apartment 3rd floor trishran nagar kha...
3,3322893,House no. 2973..prabhat colony. collector colo...
4,685152,227 Sawarbandhe Layout sawarbandhe Layout hudh...


In [6]:
data.shape

(443, 2)

In [8]:
z =data.index
for i in range(data.shape[0]):
    
    data.at[z[i],'Address' ] = data.at[z[i],'Address'].replace(","," ")
data['Address'] = data['Address'].str.lower()

In [9]:
import findspark

findspark.init()

import pyspark

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

spark = SparkSession.builder.getOrCreate()

%reload_ext sql

import sqlite3
cnn = sqlite3.connect('jupter_sql.db')

%sql sqlite:///jupter_sql.db

cursor = cnn.cursor()

cursor.execute('DROP TABLE v_address')
cnn.commit()

data.to_sql('v_address',cnn)

443

In [10]:
SQL_Query = pd.read_sql_query('''select a.customer_id as customer_id_1, b.customer_id as customer_id_2
                                    , a.address as address_1, b.address as address_2
                                 from v_address a
                                 cross join v_address b ''', cnn)

In [11]:
df = pd.DataFrame(SQL_Query)

In [12]:
df.head()

,customer_id_1,customer_id_2,address_1,address_2
0,3300924,3300924,house no. 129 suradevi bus stop kampthee road...,house no. 129 suradevi bus stop kampthee road...
1,3300924,996216,house no. 129 suradevi bus stop kampthee road...,14 mayuresh ring road behind uco bank sneh nag...
2,3300924,3063295,house no. 129 suradevi bus stop kampthee road...,darshan apartment 3rd floor trishran nagar kha...
3,3300924,3322893,house no. 129 suradevi bus stop kampthee road...,house no. 2973..prabhat colony. collector colo...
4,3300924,685152,house no. 129 suradevi bus stop kampthee road...,227 sawarbandhe layout sawarbandhe layout hudh...


In [13]:
df.shape

(196249, 4)

In [14]:
df = df[df['customer_id_1'] != df['customer_id_2']]
df.shape

(195800, 4)

In [15]:
import numpy as np
df['address_1'] = df['address_1'].replace(np.nan,' ')
df['address_2'] = df['address_2'].replace(np.nan,' ')

In [16]:
df['address_1'].head()

1    house no. 129 suradevi bus stop  kampthee road...
2    house no. 129 suradevi bus stop  kampthee road...
3    house no. 129 suradevi bus stop  kampthee road...
4    house no. 129 suradevi bus stop  kampthee road...
5    house no. 129 suradevi bus stop  kampthee road...
Name: address_1, dtype: object

In [17]:
df['address_2'].head()

1    14 mayuresh ring road behind uco bank sneh nag...
2    darshan apartment 3rd floor trishran nagar kha...
3    house no. 2973..prabhat colony. collector colo...
4    227 sawarbandhe layout sawarbandhe layout hudh...
5    plot no. 105  road no. 5 saptagiri lay out  ng...
Name: address_2, dtype: object

In [18]:
df.columns

Index(['customer_id_1', 'customer_id_2', 'address_1', 'address_2'], dtype='object')

In [19]:
a = int(df.shape[0])
df = df[:int(a/2)]

In [20]:
df['Array']= df['address_1' ].str.split(" ")
df['Array'] = np.array(df['Array'])

In [21]:
z = df.index
for i in range(df.shape[0]):
    df.at[z[i],'address_2' ]=str(df.at[z[i],'address_2' ]).lower()

In [22]:
#### Bag of words Logic ####
df.reset_index
z = df.index
for i in range(1,df.shape[0]):
    b = 0
    for j in range(len(df.at[z[i],'Array'])):
        if (df.at[z[i],'Array'][j]) in df.at[z[i],'address_2']:
            b = b+1
        else:
            pass
        df.at[z[i],'Answer'] = (b/len(df.at[z[i],'Array']))*100

In [23]:
df

,customer_id_1,customer_id_2,address_1,address_2,Array,Answer
1,3300924,996216,house no. 129 suradevi bus stop kampthee road...,14 mayuresh ring road behind uco bank sneh nag...,"[house, no., 129, suradevi, bus, stop, , kampt...",NaN
2,3300924,3063295,house no. 129 suradevi bus stop kampthee road...,darshan apartment 3rd floor trishran nagar kha...,"[house, no., 129, suradevi, bus, stop, , kampt...",17.647059
3,3300924,3322893,house no. 129 suradevi bus stop kampthee road...,house no. 2973..prabhat colony. collector colo...,"[house, no., 129, suradevi, bus, stop, , kampt...",41.176471
4,3300924,685152,house no. 129 suradevi bus stop kampthee road...,227 sawarbandhe layout sawarbandhe layout hudh...,"[house, no., 129, suradevi, bus, stop, , kampt...",29.411765
5,3300924,3049338,house no. 129 suradevi bus stop kampthee road...,plot no. 105 road no. 5 saptagiri lay out ng...,"[house, no., 129, suradevi, bus, stop, , kampt...",35.294118
...,...,...,...,...,...,...
98118,513329,466084,adarsh nagar wadi plot no 43 khadgoan road w...,14 subhash nagar joshi layout nagpur maharash...,"[adarsh, nagar, wadi, , plot, no, 43, , khadgo...",46.666667
98119,513329,868275,adarsh nagar wadi plot no 43 khadgoan road w...,2049 gopal nagar 3rd bus stop parsodi nr trish...,"[adarsh, nagar, wadi, , plot, no, 43, , khadgo...",53.333333
98120,513329,3179174,adarsh nagar wadi plot no 43 khadgoan road w...,pardi plot no 19 pardi nagpur maharashtra 440035,"[adarsh, nagar, wadi, , plot, no, 43, , khadgo...",46.666667
98121,513329,2817306,adarsh nagar wadi plot no 43 khadgoan road w...,plat no 12 balaji nagarr old kamptee road kal...,"[adarsh, nagar, wadi, , plot, no, 43, , khadgo...",56.666667


In [24]:
df = df[df['Answer']>60]

In [25]:
df.shape

(2074, 6)

In [26]:
a = write_to_gsheet(r"C:\Users\Goutham Patchipulusu\keys.json",'1r8RzjvCB4VLzYr9nVSO71R3vwbmMVPpEHkLKx2gBrmE',"Tester1",df)

C:\Users\Goutham Patchipulusu\anaconda3\lib\site-packages\pygsheets\worksheet.py:1366: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('unicode').replace('<NA>', nan)


In [37]:
# a = write_to_gsheet(r"C:\Users\Goutham Patchipulusu\keys.json",'1r8RzjvCB4VLzYr9nVSO71R3vwbmMVPpEHkLKx2gBrmE',"metric",pd.DataFrame(df['address_1']))

In [17]:
# a = int(df.shape[0])
# df1 = df[1:int(a/12)]
# df2 = df[int(a/12):int(2/*a/12)]
# df3 = df[int(2*a/12):int(3*a/12)]
# df4 = df[int(3*a/12):int(4*a/12)]
# df5 = df[int(4*a/12):int(5*a/12)]
# df6 = df[int(5*a/12):int(6*a/12)]
# df1 = df[int(6*a/12):int(7*a/12)]
# df2 = df[int(7*a/12):int(8*a/12)]
# df3 = df[int(8*a/12):int(9*a/12)]
# df4 = df[int(9*a/12):int(10*a/6)]
# df5 = df[int(10*a/12):int(11*a/12)]
# df6 = df[int(11*a/12):int(12*a/12)]

In [18]:
# #### Fuzzy Logic ####
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process

# z =df1.index
# for i in range(0,df.shape[0]):
#     df1['Fuzzy Score'] = fuzz.token_sort_ratio(str(df1['address_1']),str(df1['address_2']))
# df1

C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Fuzzy Score'] = fuzz.token_sort_ratio(str(df1['address_1']),str(df1['address_2']))
C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Fuzzy Score'] = fuzz.token_sort_ratio(str(df1['address_1']),str(df1['address_2']))
C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7

C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Fuzzy Score'] = fuzz.token_sort_ratio(str(df1['address_1']),str(df1['address_2']))
C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Fuzzy Score'] = fuzz.token_sort_ratio(str(df1['address_1']),str(df1['address_2']))
C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7

C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Fuzzy Score'] = fuzz.token_sort_ratio(str(df1['address_1']),str(df1['address_2']))
C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Fuzzy Score'] = fuzz.token_sort_ratio(str(df1['address_1']),str(df1['address_2']))
C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7

C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Fuzzy Score'] = fuzz.token_sort_ratio(str(df1['address_1']),str(df1['address_2']))
C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Fuzzy Score'] = fuzz.token_sort_ratio(str(df1['address_1']),str(df1['address_2']))
C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7

C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Fuzzy Score'] = fuzz.token_sort_ratio(str(df1['address_1']),str(df1['address_2']))
C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Fuzzy Score'] = fuzz.token_sort_ratio(str(df1['address_1']),str(df1['address_2']))
C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7

...

C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Fuzzy Score'] = fuzz.token_sort_ratio(str(df1['address_1']),str(df1['address_2']))
C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Fuzzy Score'] = fuzz.token_sort_ratio(str(df1['address_1']),str(df1['address_2']))
C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7

C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Fuzzy Score'] = fuzz.token_sort_ratio(str(df1['address_1']),str(df1['address_2']))
C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Fuzzy Score'] = fuzz.token_sort_ratio(str(df1['address_1']),str(df1['address_2']))
C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7

C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Fuzzy Score'] = fuzz.token_sort_ratio(str(df1['address_1']),str(df1['address_2']))
C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Fuzzy Score'] = fuzz.token_sort_ratio(str(df1['address_1']),str(df1['address_2']))
C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7

C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Fuzzy Score'] = fuzz.token_sort_ratio(str(df1['address_1']),str(df1['address_2']))
C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Fuzzy Score'] = fuzz.token_sort_ratio(str(df1['address_1']),str(df1['address_2']))
C:\Users\Goutham Patchipulusu\AppData\Local\Temp\ipykernel_13976\2354957459.py:7


KeyboardInterrupt



In [ ]:
# df1['Fuzzy Score']